**Modelo tipo 3 e 4**

* Tipo 3: demanda <= oferta

* Tipo 4: demanda >= oferta

* fim no porto

* sem limite transbordo

In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [2]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [3]:
df_supply = pd.read_csv('../dados/supply.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [4]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_trans_porto.shape[0]
qnt_port = df_demand.shape[0]

In [5]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,116.309766,119.994133,136.322441,106.600473,130.511560,118.331007,175.282903,125.126240,101.019697,116.005407,...,147.723937,115.106184,129.720931,105.155550,108.090037,119.630899,167.349063,136.205492,137.190180,133.324650
1,114.509039,116.641756,139.789281,109.259383,130.030495,119.066879,176.466291,122.762548,103.052422,112.882693,...,148.359951,110.830927,130.843711,102.669271,106.734392,117.541645,169.492268,139.236501,142.183116,134.882372
2,122.255078,126.066700,139.217470,110.296183,136.058529,123.322744,179.965508,131.161458,105.217732,122.085138,...,152.742272,121.030244,134.478168,111.207367,113.934943,125.628124,171.375536,139.521927,138.172298,137.802456
3,119.364718,123.619131,136.290889,107.200119,132.945463,120.126866,176.767780,128.402268,102.051908,119.558655,...,149.547199,118.978909,131.278513,108.491917,110.959856,122.802163,168.234275,136.500632,135.833592,134.614115
4,116.017654,116.454723,145.332590,114.370570,132.747652,122.820923,180.492269,123.668305,107.776005,112.974617,...,151.988631,109.495775,134.874780,103.502564,108.702545,118.748245,174.213184,144.538194,148.350978,139.228043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,122.439628,126.418733,138.819710,109.998691,136.096907,123.232204,179.812667,131.405205,104.992190,122.412653,...,152.654758,121.484155,134.341797,111.468108,114.073717,125.843128,171.118111,139.172871,137.602326,137.617301
996,112.264830,117.623943,129.315045,99.744185,125.304621,112.316125,168.976060,121.622419,94.378303,113.369682,...,141.737244,113.989192,123.475743,101.838586,103.654935,115.859998,160.631155,129.255146,130.464769,126.860730
997,122.817067,126.721717,139.256448,110.448801,136.517603,123.675964,180.262564,131.759684,105.444734,122.728448,...,153.098180,121.724988,134.790385,111.815276,114.466666,126.209172,171.570217,139.617672,137.976508,138.068968
998,116.118119,120.914383,132.943152,103.661356,129.417155,116.487253,173.120000,125.315959,98.435766,116.759445,...,145.908771,116.753964,127.629965,105.464762,107.608654,119.634751,164.643421,133.044690,133.166242,130.974440


In [6]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in M:
    demand[i] = df_demand['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]

In [7]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]
print(oferta_total, demanda_total)

278233494 559222


In [8]:
m = gp.Model("probrema")

In [9]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(k, j))

In [10]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [11]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
    for j in M:
        m.addConstr(
            (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= demand[j]
        )

else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )
    for j in M:
        m.addConstr(
            (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) == demand[j]
        )


In [12]:
for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

In [13]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [14]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6000 rows, 3000000 columns and 9000000 nonzeros
Model fingerprint: 0x7a287106
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 6e+05]
Presolve removed 3000 rows and 0 columns
Presolve time: 5.04s
Presolved: 3000 rows, 3000000 columns, 6000000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.04s

Barrier statistics:
 AA' NZ     : 3.000e+06
 Factor NZ  : 4.502e+06 (roughly 1.2 GB of memory)
 Factor Ops : 9.005e+09 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   

In [15]:
m.write("out.sol")